In [0]:
!unzip input_images.zip

In [0]:
# option 2 for images
#!unzip -uq "/content/drive/My Drive/PATH_TO_ZIP" -d "/content/drive/My Drive/Colab Notebooks/Image-Super-Resolution/input_images.zip"


In [0]:
'''
## os library is used to assign filepath locations too
import os
os.chdir('./drive/My Drive/Colab Notebooks/Image-Super-Resolution')
'''

In [1]:
'''
Colab google: uploading images in multiple subdirectories: If you would like to upload images (or files) in multiples subdirectories by using Colab google, please follow the following steps: - I'll suppose that your images(files) are split into 3 subdirectories (train, validate, test) in the main directory called (dataDir): 1- Zip the folder (dataDir) to (dataDir.zip) 2- Write this code in a Colab cell:

from google.colab import files
uploaded = files.upload()
3- Press on 'Choose Files' and upload (dataDir.zip) from your PC to the Colab Now the (dataDir.zip) is uploaded to your google drive! 4- Let us unzip the folder(dataDir.zip) to a folder called (data) by writing this simple code:

import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['dataDir.zip']), 'r')
data.extractall()
5- Now everything is ready, let us check that by printing content of (data) folder:

data.printdir()
6- Then to read the images, count them, split them and play around them, please write the following code:

train_data_dir = 'data/training'  
validation_data_dir = 'data/validation'  
test_data_dir = 'data/test' 
target_names = [item for item in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, item))]
nb_train_samples = sum([len(files) for _, _, files in os.walk(train_data_dir)])  
nb_validation_samples = sum([len(files) for _, _, files in os.walk(validation_data_dir)])
nb_test_samples = sum([len(files) for _, _, files in os.walk(test_data_dir)])
total_nb_samples = nb_train_samples + nb_validation_samples + nb_test_samples

nb_classes = len(target_names)      # number of output classes

print('Training a CNN Multi-Classifier Model ......')
print('\n - names of classes: ', target_names, '\n - # of classes: ', nb_classes)
print(' - # of trained samples: ', nb_train_samples, '\n - # of validation samples: ', nb_validation_samples,
      '\n - # of test samples: ', nb_test_samples,
       '\n - total # of samples: ', total_nb_samples, '\n - train ratio:', round(nb_train_samples/total_nb_samples*100, 2),
      '\n - validation ratio:', round(nb_validation_samples/total_nb_samples*100, 2),
      '\n - test ratio:', round(nb_test_samples/total_nb_samples*100, 2),
     ' %', '\n - # of epochs: ', epochs, '\n - batch size: ', batch_size)
7- That is it! Enjoy!
'''


SyntaxError: ignored

In [0]:
!pip install -U scipy==1.2.0

Requirement already up-to-date: scipy==1.2.0 in /usr/local/lib/python3.6/dist-packages (1.2.0)


In [0]:
from google.colab import drive
drive.mount('/content/drive/')
root_dir = "/content/drive/My Drive/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!cd drive/My Drive/Colab Notebooks/Image-Super-Resolution

/bin/bash: line 0: cd: too many arguments


In [0]:
!pwd

/content/drive/My Drive/Colab Notebooks/Image-Super-Resolution


In [0]:
import numpy as np
from scipy.misc import imsave, imread, imresize
from sklearn.feature_extraction.image import reconstruct_from_patches_2d, extract_patches_2d
from scipy.ndimage.filters import gaussian_filter

from keras import backend as K

import os
import time


In [0]:
'''
_image_scale_multiplier is a special variable which is used to alter image size.

The default image size is 32x32. If a true upscaling model is used, then the input image size is 16x16,
which not offer adequate training samples.
'''
_image_scale_multiplier = 1

img_size = 128 * _image_scale_multiplier
stride = 64 * _image_scale_multiplier

assert (img_size ** 2) % (stride ** 2) == 0, "Number of images generated from strided subsample of the image needs to be \n" \
                                             "a positive integer. Change stride such that : \n" \
                                             "(img_size ** 2) / (stride ** 2) is a positive integer."


In [0]:
input_path = r"D:\Yue\Documents\Datasets\train2014\train2014\\" # r"input_images/"
validation_path = r"val_images/"


In [0]:

validation_set5_path = validation_path + r"set5/"
validation_set14_path = validation_path + r"set14/"

base_dataset_dir = r"/content/drive/My Drive/Colab Notebooks/Image-Super-Resolution" + r"/Dataset/"

output_path = base_dataset_dir + "train_images/train/"
validation_output_path = base_dataset_dir + r"train_images/validation/"

if not os.path.exists(output_path):
    os.makedirs(output_path)


In [0]:

def transform_images_temp(directory, output_directory, scaling_factor=2, max_nb_images=-1, true_upscale=False,
                          id_advance=0):
    index = 1

    if not os.path.exists(output_directory + "X/"):
        os.makedirs(output_directory + "X/")

    if not os.path.exists(output_directory + "y/"):
        os.makedirs(output_directory + "y/")

    # For each image in input_images directory
    nb_images = len([name for name in os.listdir(directory)])

    if max_nb_images != -1:
        print("Transforming %d images." % max_nb_images)
    else:
        assert max_nb_images <= nb_images, "Max number of images must be less than number of images in path"
        print("Transforming %d images." % (nb_images))

    if nb_images == 0:
        print("Extract the training images or images from imageset_91.zip (found in the releases of the project) "
              "into a directory with the name 'input_images'")
        print("Extract the validation images or images from set5_validation.zip (found in the releases of the project) "
              "into a directory with the name 'val_images'")
        exit()

    for file in os.listdir(directory):
        img = imread(directory + file, mode='RGB')

        # Resize to 256 x 256
        img = imresize(img, (img_size, img_size))

        # Create patches
        hr_patch_size = 64
        lr_patch_size = 32
        nb_hr_images = (img_size ** 2) // (stride ** 2)

        hr_samples = np.empty((nb_hr_images, hr_patch_size, hr_patch_size, 3))

        image_subsample_iterator = subimage_generator(img, stride, hr_patch_size, nb_hr_images)

        stride_range = np.sqrt(nb_hr_images).astype(int)

        i = 0
        for j in range(stride_range):
            for k in range(stride_range):
                hr_samples[i, :, :, :] = next(image_subsample_iterator)
                i += 1


        t1 = time.time()
        # Create nb_hr_images 'X' and 'Y' sub-images of size hr_patch_size for each patch
        for i in range(nb_hr_images):
            ip = hr_samples[i]
            # Save ground truth image X
            imsave(output_directory + "/y/" + "%d_%d.png" % (index + id_advance, i + 1), ip)

            # Apply Gaussian Blur to Y
            #op = gaussian_filter(ip, sigma=0.5)

            # Subsample by scaling factor to Y
            op = imresize(ip, (lr_patch_size, lr_patch_size), interp='bicubic')

            if not true_upscale:
                # Upscale by scaling factor to Y
                op = imresize(op, (hr_patch_size, hr_patch_size), interp='bicubic')

            # Save Y
            imsave(output_directory + "/X/" + "%d_%d.png" % (index + id_advance, id_advance + i + 1), op)

        print("Finished image %d in time %0.2f seconds. (%s)" % (index + id_advance, time.time() - t1, file))
        index += 1

        if max_nb_images > 0 and index >= max_nb_images:
            print("Transformed maximum number of images. ")
            break

    print("Images transformed. Saved at directory : %s" % (output_directory))


def image_count():
    return len([name for name in os.listdir(output_path + "X/")])


def val_image_count():
    return len([name for name in os.listdir(validation_output_path + "X/")])


def subimage_generator(img, stride, patch_size, nb_hr_images):
    for _ in range(nb_hr_images):
        for x in range(0, img_size, stride):
            for y in range(0, img_size, stride):
                subimage = img[x : x + patch_size, y : y + patch_size, :]

                yield subimage


def make_patches(x, scale, patch_size, upscale=True, verbose=1):
    '''x shape: (num_channels, rows, cols)'''
    height, width = x.shape[:2]
    if upscale: x = imresize(x, (height * scale, width * scale))
    patches = extract_patches_2d(x, (patch_size, patch_size))
    return patches


def combine_patches(in_patches, out_shape, scale):
    '''Reconstruct an image from these `patches`'''
    recon = reconstruct_from_patches_2d(in_patches, out_shape)
    return recon

def image_generator(directory, scale_factor=2, target_shape=None, channels=3, small_train_images=False, shuffle=True,
                    batch_size=32, nb_inputs=1, seed=None):
    if not target_shape:
        if small_train_images:
            if K.image_dim_ordering() == "th":
                image_shape = (channels, 16 * _image_scale_multiplier, 16 * _image_scale_multiplier)
                y_image_shape = (channels, 16 * scale_factor * _image_scale_multiplier,
                                 16 * scale_factor * _image_scale_multiplier)
            else:
                # image_shape = (16 * _image_scale_multiplier, 16 * _image_scale_multiplier, channels)
                # y_image_shape = (16 * scale_factor * _image_scale_multiplier,
                #                  16 * scale_factor * _image_scale_multiplier, channels)
                image_shape = (32 * _image_scale_multiplier, 32 * _image_scale_multiplier, channels)
                y_image_shape = (32 * scale_factor * _image_scale_multiplier,
                                 32 * scale_factor * _image_scale_multiplier, channels)
        else:
            if K.image_dim_ordering() == "th":
                image_shape = (channels, 32 * scale_factor * _image_scale_multiplier, 32 * scale_factor * _image_scale_multiplier)
                y_image_shape = image_shape
            else:
                image_shape = (32 * scale_factor * _image_scale_multiplier, 32 * scale_factor * _image_scale_multiplier,
                               channels)
                y_image_shape = image_shape
    else:
        if small_train_images:
            if K.image_dim_ordering() == "th":
                y_image_shape = (3,) + target_shape

                target_shape = (target_shape[0] * _image_scale_multiplier // scale_factor,
                                target_shape[1] * _image_scale_multiplier // scale_factor)
                image_shape = (3,) + target_shape
            else:
                y_image_shape = target_shape + (channels,)

                target_shape = (target_shape[0] * _image_scale_multiplier // scale_factor,
                                target_shape[1] * _image_scale_multiplier // scale_factor)
                image_shape = target_shape + (channels,)
        else:
            if K.image_dim_ordering() == "th":
                image_shape = (channels,) + target_shape
                y_image_shape = image_shape
            else:
                image_shape = target_shape + (channels,)
                y_image_shape = image_shape

    file_names = [f for f in sorted(os.listdir(directory + "X/"))]
    X_filenames = [os.path.join(directory, "X", f) for f in file_names]
    y_filenames = [os.path.join(directory, "y", f) for f in file_names]

    nb_images = len(file_names)
    print("Found %d images." % nb_images)

    index_generator = _index_generator(nb_images, batch_size, shuffle, seed)

    while 1:
        index_array, current_index, current_batch_size = next(index_generator)

        batch_x = np.zeros((current_batch_size,) + image_shape)
        batch_y = np.zeros((current_batch_size,) + y_image_shape)

        for i, j in enumerate(index_array):
            x_fn = X_filenames[j]
            img = imread(x_fn, mode='RGB')
            if small_train_images:
                img = imresize(img, (32 * _image_scale_multiplier, 32 * _image_scale_multiplier))
            img = img.astype('float32') / 255.

            if K.image_dim_ordering() == "th":
                batch_x[i] = img.transpose((2, 0, 1))
            else:
                batch_x[i] = img

            y_fn = y_filenames[j]
            img = imread(y_fn, mode="RGB")
            img = img.astype('float32') / 255.

            if K.image_dim_ordering() == "th":
                batch_y[i] = img.transpose((2, 0, 1))
            else:
                batch_y[i] = img

        if nb_inputs == 1:
            yield (batch_x, batch_y)
        else:
            batch_x = [batch_x for i in range(nb_inputs)]
            yield batch_x, batch_y

In [0]:
def _index_generator(N, batch_size=32, shuffle=True, seed=None):
    batch_index = 0
    total_batches_seen = 0

    while 1:
        if seed is not None:
            np.random.seed(seed + total_batches_seen)

        if batch_index == 0:
            index_array = np.arange(N)
            if shuffle:
                index_array = np.random.permutation(N)

        current_index = (batch_index * batch_size) % N

        if N >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1
        else:
            current_batch_size = N - current_index
            batch_index = 0
        total_batches_seen += 1

        yield (index_array[current_index: current_index + current_batch_size],
               current_index, current_batch_size)


In [0]:
def smooth_gan_labels(y):
    assert len(y.shape) == 2, "Needs to be a binary class"
    y = np.asarray(y, dtype='int')
    Y = np.zeros(y.shape, dtype='float32')

    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if y[i, j] == 0:
                Y[i, j] = np.random.uniform(0.0, 0.3)
            else:
                Y[i, j] = np.random.uniform(0.7, 1.2)

    return Y



In [0]:
if __name__ == "__main__":
    # Transform the images once, then run the main code to scale images

    # Change scaling factor to increase the scaling factor
    scaling_factor = 2

    # Set true_upscale to True to generate smaller training images that will then be true upscaled.
    # Leave as false to create same size input and output images
    true_upscale = True

    # transform_images_temp(input_path, output_path, scaling_factor=scaling_factor, max_nb_images=-1,
    #                  true_upscale=true_upscale)
    transform_images_temp(validation_set14_path, validation_output_path, scaling_factor=scaling_factor, max_nb_images=-1,
                     true_upscale=true_upscale)
    # transform_images_temp(validation_set14_path, validation_output_path, scaling_factor=scaling_factor, max_nb_images=-1,
    #                       true_upscale=true_upscale)
    pass


Transforming 14 images.
Finished image 1 in time 0.04 seconds. (barbara.bmp)
Finished image 2 in time 0.06 seconds. (coastguard.bmp)
Finished image 3 in time 0.04 seconds. (bridge.bmp)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWar

Finished image 4 in time 0.06 seconds. (baboon.bmp)
Finished image 5 in time 0.04 seconds. (zebra.bmp)
Finished image 6 in time 0.05 seconds. (lenna.bmp)
Finished image 7 in time 0.06 seconds. (man.bmp)
Finished image 8 in time 0.05 seconds. (flowers.bmp)
Finished image 9 in time 0.06 seconds. (pepper.bmp)
Finished image 10 in time 0.04 seconds. (monarch.bmp)
Finished image 11 in time 0.06 seconds. (ppt3.bmp)
Finished image 12 in time 0.04 seconds. (face.bmp)
Finished image 13 in time 0.04 seconds. (foreman.bmp)
Finished image 14 in time 0.06 seconds. (comic.bmp)
Images transformed. Saved at directory : /content/drive/My Drive/Colab Notebooks/Image-Super-Resolution/Dataset/train_images/validation/


In [0]:
!pip install opencv

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [0]:
!pip install -q keras
import keras

In [0]:
import models
import argparse
import tensorflow as tf

In [0]:
import models
import argparse
import tensorflow as tf

parser = argparse.ArgumentParser(description="Up-Scales an image using Image Super Resolution Model")
parser.add_argument("imgpath", type=str, nargs="+", help="Path to input image")
parser.add_argument("--model", type=str, default="distilled_rnsr", help="Use either image super resolution (sr), "
                        "expanded super resolution (esr), denoising auto encoder sr (dsr), "
                        "deep denoising sr (ddsr) or res net sr (rnsr)")
parser.add_argument("--scale", default=2, help='Scaling factor. Default = 2x')
parser.add_argument("--mode", default="patch", type=str, help='Mode of operation. Choices are "fast" or "patch"')
parser.add_argument("--save_intermediate", dest='save', default='True', type=str,
                        help="Whether to save bilinear upscaled image")
parser.add_argument("--suffix", default="scaled", type=str, help='Suffix of saved image')
parser.add_argument("--patch_size", type=int, default=8, help='Patch Size')

def strToBool(v):
    return v.lower() in ("true", "yes", "t", "1")

args = parser.parse_args()


suffix = args.suffix

model_type = str(args.model).lower()
if not model_type in ["sr", "esr", "dsr", "ddsr", "rnsr", "distilled_rnsr"]:
    raise ValueError('Model type must be either "sr", "esr", "dsr", '
                     '"ddsr", "rnsr" or "distilled_rnsr"')

mode = str(args.mode).lower()
assert mode in ['fast', 'patch'], 'Mode of operation must be either "fast" or "patch"'

scale_factor = int(args.scale)
save = strToBool(args.save)

patch_size = int(args.patch_size)
assert patch_size > 0, "Patch size must be a positive integer"

with tf.device('/CPU:0'):
    path = args.imgpath
    for p in path:
        if model_type == "sr":
            model = models.ImageSuperResolutionModel(scale_factor)
        elif model_type == "esr":
            model = models.ExpantionSuperResolution(scale_factor)
        elif model_type == "dsr":
            model = models.DenoisingAutoEncoderSR(scale_factor)
        elif model_type == "ddsr":
            model = models.DeepDenoiseSR(scale_factor)
        elif model_type == "rnsr":
            model = models.ResNetSR(scale_factor)
        elif model_type == "distilled_rnsr":
            model = models.DistilledResNetSR(scale_factor)
        else:
            model = models.DistilledResNetSR(scale_factor)

        model.upscale(p, save_intermediate=save, mode=mode, patch_size=patch_size, suffix=suffix)

In [0]:
!ls

In [0]:
cd drive

In [0]:
ls

In [0]:
cd My Drive


In [0]:
ls